<a href="https://colab.research.google.com/github/ueponx/20230405-git-workshop/blob/main/GetNagoyaEvents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**必要なライブラリをインストール**

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas

**HTMLを解析して、CSV化する**

1.   実行時にはURLを入力するインプットボックスが表示
2.   実行後にevent.csvがダウンロード可能になる




In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
import io
import re

def extract_event_info(date_tag):
    """
    HTMLの日付タグからイベント情報を抽出する関数。

    :param date_tag: BeautifulSoupオブジェクトの日付タグ
    :return: イベント情報のリスト（辞書形式）
    """
    date_text = date_tag.get_text(strip=True)
    day, weekday = date_text[:-3], date_text[-3:]
    weekday_dict = {'月曜日': '月', '火曜日': '火', '水曜日': '水', '木曜日': '木', '金曜日': '金', '土曜日': '土', '日曜日': '日'}
    date = f"{day}({weekday_dict.get(weekday, '?')})"
    events = []
    events_td = date_tag.find_next_sibling('td')
    if events_td:
        for li in events_td.find_all('li'):
            event_title_tag = li.find('a')
            event_title = event_title_tag.get_text(strip=True) if event_title_tag else "タイトル不明"
            event_url = event_title_tag['href'] if event_title_tag and event_title_tag.has_attr('href') else "URL不明"
            event_categories = [span.text for span in li.find_all('span', class_=re.compile('eve_cate'))]
            events.append({
                '日付': date,
                'イベントタイトル': event_title,
                'イベントURL': event_url,
                'カテゴリ': ','.join(event_categories)
            })
    return events

def parse_html_from_url(url):
    """
    URLからHTMLを取得し、イベント情報を抽出する関数。

    :param url: イベント情報が含まれるウェブページのURL
    :return: イベント情報のリスト（辞書形式）
    """
    response = requests.get(url)
    response.raise_for_status()
    response.encoding = 'utf-8'  # エンコーディングをUTF-8に設定
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    date_tags = soup.find_all('th', {'scope': 'row', 'class': 'cal_date'})
    all_events = []
    for date_tag in date_tags:
        all_events.extend(extract_event_info(date_tag))
    return all_events

def save_to_csv(events):
    """
    イベント情報をCSVファイルに保存してダウンロードする関数。

    :param events: イベント情報のリスト（辞書形式）
    """
    df = pd.DataFrame(events)
    # CSVデータをUTF-8 BOMエンコーディングで文字列として取得
    csv_content = df.to_csv(index=False, quoting=1, encoding='utf-8-sig')
    # 文字列をバイト列に変換し、BytesIOオブジェクトに変換
    csv_buffer = io.BytesIO(csv_content.encode('utf-8-sig'))
    # バッファの内容をファイルに保存
    with open('events.csv', 'wb') as f:
        f.write(csv_buffer.getvalue())
    # ファイルをダウンロード
    files.download('events.csv')

if __name__ == "__main__":
    url = input("イベント情報が含まれるウェブページのURLを入力してください: ")
    events = parse_html_from_url(url)
    save_to_csv(events)
    print("イベント情報をCSVファイルとしてダウンロードしました。")

イベント情報が含まれるウェブページのURLを入力してください: https://www.city.nagoya.jp/main/event2/curr.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

イベント情報をCSVファイルとしてダウンロードしました。
